In [ ]:
### DRAFTS

def normalized_entropy(counts, logfun=lambda x: log(x)):
    total = float(sum(counts.values()))
    probs = [counts[k]/total for k in counts]
    probs_max = [1/float(len(counts))]*int(len(counts))
    ent = -sum([p*logfun(p) for p in probs])
    ent_max = -sum([p*logfun(p) for p in probs_max])
    if ent_max != 0:
        return ent / ent_max
    else:
        return -1.0
    
def Cosine_between_Years(past,present):
    intersection = set(past.keys()) & set(present.keys())
    numerator = sum([past[x] * present[x] for x in intersection])
    
    sum1 = sum([past[x]**2 for x in past.keys()])
    sum2 = sum([present[x]**2 for x in present.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    
def counter_cosine_similarity(v1, v2):
    terms = set(v1).union(v2)
    dotprod = sum(v1.get(k, 0) * v2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(v1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(v2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)



# Abstract Text Entropy

Entropy_list = []
for year_ in range(int(minimum_year),maximum_year): 

    try:
        t_list = abstract_df.query("publication_year==@year_")['Text']
        t_counter = Counter(x for xs in t_list.tolist() for x in set(xs))
        t_counter = Counter({k:v for k, v in dict(t_counter).items() if v > 1})

        print(str(year_))
        if year_ == int(minimum_year):
            compare_vocab_size = len(t_counter)
            t_past = t_counter
            cosine_all = cosine_T500 = cosine_MinYearBaseline = 0
        else:
            cosine_all = Cosine_between_Years(t_past,t_counter)
            cosine_T500 = Cosine_between_Years(Counter(t_past.most_common(500)),Counter(t_counter.most_common(500)))
            cosine_MinYearBaseline= Cosine_between_Years(Counter(t_past.most_common(compare_vocab_size)),Counter(t_counter.most_common(compare_vocab_size)))
            t_past = t_counter
            
        Entropy_list.append({"Year":year_,\
                             "Consensus_All":(1-normalized_entropy(t_counter)),\
                             "Consensus_T500":(1-normalized_entropy(dict(t_counter.most_common(500)))),\
                             "Consensus_MinYearBaseline":(1-normalized_entropy(dict(t_counter.most_common(compare_vocab_size)))),\
                             "Cosine_Year_All":cosine_all,\
                             "Cosine_Year_T500":cosine_T500,\
                             "Cosine_Year_MinYearBaseline":cosine_MinYearBaseline,\
                             "Vocab_Num":len(t_counter)})

    except Exception as e:
        print(e)
        next

Entropy_df = pd.DataFrame(Entropy_list)

Entropy_df["Year"] = Entropy_df["Year"].astype(int)

# Final Output

Full_Degree_Entropy_df = pd.merge(Entropy_df,Degree_df,on=["Year"])
Full_Degree_Entropy_df = pd.merge(Full_Degree_Entropy_df,t4,on=["Year"])
Full_Degree_Entropy_df["Discipline"] = discipline

Full_Degree_Entropy_df.to_csv("Project_Hierarchy_and_Consensus/OUTPUT_Python_OpenAlex_Hierarchy_and_Consensus_"+str(discipline.split("https://openalex.org/")[1])+".csv.gz",index=False,compression="gzip")